In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Extract Multiple features from a directory

In [ ]:
%matplotlib inline
import PIL
import tqdm
import pickle
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets
from skimage.filters import gabor
from skimage.feature import local_binary_pattern

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import os
import numpy as np
import pandas as pd
from skimage import io, measure, color
from skimage.filters import threshold_otsu
from skimage.feature import local_binary_pattern
from skimage.filters import gabor
from skimage.measure import shannon_entropy
from PIL import Image
from scipy.stats import kurtosis, skew

# Directory and paths
dirr1 = '/content/drive/MyDrive/Computer Vision and Machine Learning for Medical Data Analysis/Module-3/ROI/malignant'  # Source folder
csv = '/content/malignant.csv'  # Path to save CSV
c = 1  # Class label (0 or 1)

# Initialize DataFrame
df1 = pd.DataFrame(columns=['class', 'Area', 'ConvexArea', 'Solidity',
                            'Shannon_Entropy', 'lbp_energy', 'lbp_entropy', 'gabor_energy', 'gabor_entropy'])

black = 0
j = 0
onlyfiles = next(os.walk(dirr1))[2]
print(f'Total images: {len(onlyfiles)}')

# Process each image
for i in os.listdir(dirr1):
    path = os.path.join(dirr1, i)
    print(f'Working on image: {i}')
    try:
        # Calculate Area
        image = io.imread(path, as_gray=True)
        thresh = threshold_otsu(image)
        binary = image > thresh
        area = np.sum(binary)

        # Calculate Convex Area and Solidity
        props = measure.regionprops(binary.astype(int))
        convex_area = round(props[0].convex_area, 4) if props else 0
        solidity = round(area / convex_area, 4) if convex_area != 0 else 0

        # Shannon Entropy
        sha_entropy = round(shannon_entropy(image), 4)

        # LBP Features
        feat_lbp = local_binary_pattern(image, P=8, R=1, method='uniform')
        lbp_hist, _ = np.histogram(feat_lbp, bins=8, range=(0, 8))
        lbp_prob = lbp_hist / np.sum(lbp_hist)
        lbp_energy = np.sum(lbp_prob ** 2)
        lbp_entropy = -np.sum(lbp_prob * np.log2(lbp_prob + 1e-9))

        # Gabor Features
        gabor_real, gabor_imag = gabor(image, frequency=0.6)
        gabor_magnitude = (gabor_real**2 + gabor_imag**2) / 2
        gabor_hist, _ = np.histogram(gabor_magnitude, bins=8)
        gabor_prob = gabor_hist / np.sum(gabor_hist)
        gabor_energy = np.sum(gabor_prob ** 2)
        gabor_entropy = -np.sum(gabor_prob * np.log2(gabor_prob + 1e-9))

        # Append to DataFrame
        df1.loc[j] = {
            'class': c,
            'Area': area,
            'ConvexArea': convex_area,
            'Solidity': solidity,
            'Shannon_Entropy': sha_entropy,
            'lbp_energy': lbp_energy,
            'lbp_entropy': lbp_entropy,
            'gabor_energy': gabor_energy,
            'gabor_entropy': gabor_entropy
        }
        j += 1

    except Exception as e:
        print(f'Error processing image {i}: {e}')
        black += 1
        continue

print(f'Black images found: {black} out of total images: {len(onlyfiles)}')

# Save to CSV
df1.to_csv(csv, index=False)
print('Feature extraction complete. Data saved to CSV.')


Total images: 1
Working on image: segmented_image.png
Black images found: 0 out of total images: 1
Feature extraction complete. Data saved to CSV.


In [ ]:
df = pd.read_csv('/content/malignant.csv')
df

,class,Area,ConvexArea,Solidity,Shannon_Entropy,lbp_energy,lbp_entropy,gabor_energy,gabor_entropy
0,1,38643,88477.0,0.4368,1.487,0.806109,0.731198,0.998194,0.011589
